In [2]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [4]:
sql = """
    select * from apl.damir_age_stats_2 
    where (psp_spe_snds > 0 and psp_spe_snds <= 42) or (psp_act_snds > 0 and psp_act_snds <= 55)
"""
damir = pd.read_sql(sql, config.connection_string)
damir

,psp_spe_snds,psp_act_snds,ann,age_ben_snds,nb,total_nb,ratio
0,1.0,NaN,24,30,1,4,0.25
1,1.0,NaN,24,60,2,4,0.50
2,1.0,NaN,24,70,1,4,0.25
3,7.0,NaN,24,30,1,1,1.00
4,14.0,NaN,24,50,1,1,1.00
5,15.0,NaN,24,80,1,1,1.00
6,33.0,NaN,24,40,1,1,1.00
7,NaN,19.0,24,60,1,1,1.00


In [3]:
sql =f"""
    select year, sum(pop) pop, sum(pop0002)/sum(pop) p0002, sum(pop0305)/sum(pop) p0305, sum(pop0610)/sum(pop) p0610, sum(pop1117)/sum(pop) p1117, sum(pop1824)/sum(pop) p1824, sum(pop2539)/sum(pop) p2539, sum(pop4054)/sum(pop) p4054, sum(pop5564)/sum(pop) p5564, sum(pop6579)/sum(pop) p6579, sum(pop80p)/sum(pop) p80p
    from iris.pop_iris
    group by year
"""
pop_iris = pd.read_sql(sql, config.connection_string)
pop_iris

,year,pop,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,67162154.0,0.032056,0.034700,0.061375,0.086881,0.080607,0.180414,0.195735,0.126311,0.140958,0.060963
1,21,67408052.0,0.031403,0.033951,0.060709,0.087117,0.081043,0.179629,0.194405,0.126668,0.144234,0.060842


In [4]:
pop_iris[pop_iris["year"]==20].iloc[0].to_dict()

{'year': 20.0,
 'pop': 67162153.99999976,
 'p0002': 0.03205633570254455,
 'p0305': 0.034699653839599026,
 'p0610': 0.061374556855644094,
 'p1117': 0.08688149205273803,
 'p1824': 0.08060697124120396,
 'p2539': 0.18041440053576946,
 'p4054': 0.1957352897080859,
 'p5564': 0.12631058481640903,
 'p6579': 0.14095808265358695,
 'p80p': 0.060962632594421036}

In [5]:
def compute_overrepresentation(damir_dataset, year, psp_spe_snds, psp_act_snds):
    # yy=20 if year==20 else 21
    yy=min(year,21)
    props=pop_iris[pop_iris["year"]==yy].iloc[0].to_dict()
    bb = 1 if psp_spe_snds==1 or psp_spe_snds==12 else 0
    damir = damir_dataset[damir_dataset["ann"]==year]
    if psp_spe_snds is not None:
        damir = damir[damir["psp_spe_snds"]==psp_spe_snds]
    else:
        damir = damir[damir["psp_act_snds"]==psp_act_snds]
    overs={"year":year, "psp_spe_snds":psp_spe_snds, "psp_act_snds":psp_act_snds}
    overs["p0002"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(3+bb*3)/20/(props["p0002"])).iloc[0]
    overs["p0305"]=(damir[damir["age_ben_snds"]==0]["ratio"]*3/20/(props["p0305"])).iloc[0]
    overs["p0610"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(5-bb)/20/(props["p0610"])).iloc[0]
    overs["p1117"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(7-bb)/20/(props["p1117"])).iloc[0]
    overs["p1824"]=((damir[damir["age_ben_snds"]==0]["ratio"]*(2-bb)/20+damir[damir["age_ben_snds"]==20]["ratio"].iloc[0]*5/10)/(props["p1824"])).iloc[0]
    overs["p2539"]=((damir[damir["age_ben_snds"]==20]["ratio"]*5/10+damir[damir["age_ben_snds"]==30]["ratio"].iloc[0]*10/10)/(props["p2539"])).iloc[0]
    overs["p4054"]=((damir[damir["age_ben_snds"]==40]["ratio"]*10/10+damir[damir["age_ben_snds"]==50]["ratio"].iloc[0]*5/10)/(props["p4054"])).iloc[0]
    overs["p5564"]=((damir[damir["age_ben_snds"]==50]["ratio"]*5/10+damir[damir["age_ben_snds"]==60]["ratio"].iloc[0]*5/10)/(props["p5564"])).iloc[0]
    overs["p6579"]=((damir[damir["age_ben_snds"]==60]["ratio"]*5/10+damir[damir["age_ben_snds"]==70]["ratio"].iloc[0]*10/10)/(props["p6579"])).iloc[0]
    overs["p80p"]=((damir[damir["age_ben_snds"]==80]["ratio"]*10/10+damir[damir["age_ben_snds"]==99]["ratio"].iloc[0]*10/10)/(props["p80p"])).iloc[0]
    if psp_spe_snds == 12:
        overs["p0002"]+=overs["p1117"]
        overs["p0002"]+=overs["p1824"]
        overs["p0002"]+=overs["p2539"]
        overs["p0002"]+=overs["p4054"]
        overs["p0002"]+=overs["p5564"]
        overs["p0002"]+=overs["p6579"]
        overs["p0002"]+=overs["p80p"]
        overs["p1117"]=overs["p1824"]=overs["p2539"]=overs["p4054"]=overs["p5564"]=overs["p6579"]=overs["p80p"]=0
    return overs
    
overs = compute_overrepresentation(damir, 20, 1, None)
overs

{'year': 20,
 'psp_spe_snds': 1,
 'p0002': 0.9053864732849047,
 'p0305': 0.4182083899500245,
 'p0610': 0.3152595496146517,
 'p1117': 0.3340570246025043,
 'p1824': 0.6107626344160559,
 'p2539': 0.7861679204980528,
 'p4054': 0.8855176134587468,
 'p5564': 1.133266417316728,
 'p6579': 1.6382867998597272,
 'p80p': 2.7824348936889196}

In [6]:
def df_append_dict(df, dico):
    dico_df = pd.DataFrame([dico])
    return pd.concat([df, dico_df], ignore_index=True)

df = pd.DataFrame(columns=overs.keys())
df = df_append_dict(df, overs)
df

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,1,0.905386,0.418208,0.31526,0.334057,0.610763,0.786168,0.885518,1.133266,1.638287,2.782435


In [7]:
years=damir["ann"].unique()
print(years)
psp_spe_sndss=damir["psp_spe_snds"].unique().dropna()
psp_act_sndss=damir["psp_act_snds"].unique().dropna()
psp_spe_sndss, psp_act_sndss

[20 21 22 23 24]


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 18, 31, 32,
       33, 34, 35, 36, 37, 38, 42], dtype=int64)

In [8]:
df = pd.DataFrame(columns=overs.keys())
for year in years:
    for i in psp_spe_sndss:
        overs = compute_overrepresentation(damir, year, i, None)
        df = df_append_dict(df, overs)
    for i in psp_act_sndss:
        overs = compute_overrepresentation(damir, year, None, i)
        df = df_append_dict(df, overs)
df

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,1,0.905386,0.418208,0.315260,0.334057,0.610763,0.786168,0.885518,1.133266,1.638287,2.782435
1,20,2,0.359383,0.332007,0.312847,0.309401,0.603956,0.815675,0.985486,1.296469,1.841326,1.949265
2,20,3,0.122765,0.113413,0.106868,0.105691,0.240940,0.376122,0.802847,1.401387,2.448984,3.440615
3,20,4,0.319696,0.295343,0.278299,0.275233,0.604697,0.772066,1.020762,1.355475,1.847434,1.952917
4,20,5,0.579068,0.534957,0.504085,0.498532,0.870782,0.909465,0.880232,1.082622,1.578259,2.015069
...,...,...,...,...,...,...,...,...,...,...,...,...
115,24,35,0.121483,0.112365,0.104731,0.102177,0.293898,0.508949,0.872671,1.313280,2.314966,3.155235
116,24,36,3.371749,3.118690,2.906815,2.835922,1.304666,0.525829,0.386129,0.306373,0.280972,0.166382
117,24,37,0.170899,0.158072,0.147333,0.143740,0.716997,1.083676,1.150167,1.295029,1.604101,1.578351
118,24,38,0.369621,0.341880,0.318654,0.310882,0.622499,0.978948,0.940986,1.180621,1.715211,2.041186


In [9]:
# df.to_sql("overrepresentation", config.connection_string, schema="apl", if_exists="replace", index=False)

120